In [2]:
# Import required libraries
import pandas as pd
from sqlalchemy import create_engine

# Setup MySQL connection
user = 'root'
password = 'password_saya'  # Replace with your MySQL password
host = 'localhost'
database = 'commercentx'

# Create the engine
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')
conn = engine.connect()


In [3]:
# Load the CSV file into a Pandas DataFrame
csv_file = 'ecommerce-session-bigquery.csv'
data = pd.read_csv(csv_file)

In [4]:
# Display basic information about the dataset
print("Basic Info:")
print(data.info())

Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   fullVisitorId            10000 non-null  uint64 
 1   channelGrouping          10000 non-null  object 
 2   time                     10000 non-null  int64  
 3   country                  10000 non-null  object 
 4   city                     10000 non-null  object 
 5   totalTransactionRevenue  619 non-null    float64
 6   transactions             628 non-null    float64
 7   timeOnSite               9713 non-null   float64
 8   pageviews                9999 non-null   float64
 9   sessionQualityDim        19 non-null     float64
 10  date                     10000 non-null  int64  
 11  visitId                  10000 non-null  int64  
 12  type                     10000 non-null  object 
 13  productRefundAmount      0 non-null      float64
 14  productQuan

In [5]:
# Show summary statistics of the dataset (including categorical columns)
print("\nSummary Statistics:")
print(data.describe(include='all'))


Summary Statistics:
        fullVisitorId channelGrouping          time        country  \
count    1.000000e+04           10000  1.000000e+04          10000   
unique            NaN               8           NaN            106   
top               NaN  Organic Search           NaN  United States   
freq              NaN            4539           NaN           6880   
mean     4.538050e+18             NaN  3.248462e+05            NaN   
std      3.038982e+18             NaN  5.647221e+05            NaN   
min      1.664653e+14             NaN  0.000000e+00            NaN   
25%      1.701094e+18             NaN  4.252400e+04            NaN   
50%      4.449181e+18             NaN  1.357180e+05            NaN   
75%      7.170959e+18             NaN  3.542545e+05            NaN   
max      9.994767e+18             NaN  1.089403e+07            NaN   

                                 city  totalTransactionRevenue  transactions  \
count                           10000             6.190000

In [6]:
# Check for missing values
print("\nMissing Values:")
print(data.isnull().sum())


Missing Values:
fullVisitorId                  0
channelGrouping                0
time                           0
country                        0
city                           0
totalTransactionRevenue     9381
transactions                9372
timeOnSite                   287
pageviews                      1
sessionQualityDim           9981
date                           0
visitId                        0
type                           0
productRefundAmount        10000
productQuantity             9955
productPrice                   0
productRevenue             10000
productSKU                     0
v2ProductName                  0
v2ProductCategory              0
productVariant                 0
currencyCode                 386
itemQuantity               10000
itemRevenue                10000
transactionRevenue         10000
transactionId              10000
pageTitle                     92
searchKeyword              10000
pagePathLevel1                 0
eCommerceAction_type      

In [7]:
# Check for duplicated rows
print("\nDuplicated Rows:")
print(data.duplicated().sum())


Duplicated Rows:
0


In [8]:
# Show the first few rows to visually inspect the data
print("\nSample Data:")
data.head()


Sample Data:


,fullVisitorId,channelGrouping,time,country,city,totalTransactionRevenue,transactions,timeOnSite,pageviews,sessionQualityDim,...,itemQuantity,itemRevenue,transactionRevenue,transactionId,pageTitle,searchKeyword,pagePathLevel1,eCommerceAction_type,eCommerceAction_step,eCommerceAction_option
0,2515546493837534633,Organic Search,966564,Taiwan,(not set),NaN,NaN,1567.0,82.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
1,9361741997835388618,Organic Search,157377,France,not available in demo dataset,NaN,NaN,321.0,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
2,7313828956068851679,Referral,228279,United States,San Francisco,NaN,NaN,927.0,11.0,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
3,6036794406403793540,Organic Search,1615618,United States,Boulder,NaN,NaN,1616.0,13.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
4,7847280609739507227,Organic Search,37832,Canada,not available in demo dataset,NaN,NaN,1222.0,45.0,53.0,...,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN


In [9]:
# Identify columns with uint64 data type
columns_to_convert = ['fullVisitorId', 'productRevenue']  # Modify based on your dataset

# Convert uint64 columns to object (string) type
for col in columns_to_convert:
    if col in data.columns:
        data[col] = data[col].astype(str)

# Upload DataFrame to MySQL table
data.to_sql('ecommerce_sessions', conn, index=False, if_exists='replace')

print("Data uploaded successfully")

Data uploaded successfully


In [10]:
# Upload the DataFrame to MySQL table (replace 'ecommerce_sessions' with your table name)
data.to_sql('ecommerce_sessions', conn, index=False, if_exists='replace')

10000

In [11]:
# Query for Test Case 1: Channel Analysis
query = '''
WITH Country_Revenue AS (
    SELECT 
        country,
        SUM(CAST(totalTransactionRevenue AS UNSIGNED)) AS total_country_revenue
    FROM ecommerce_sessions
    WHERE totalTransactionRevenue IS NOT NULL
    GROUP BY country
    ORDER BY total_country_revenue DESC
    LIMIT 5
)

SELECT 
    e.channelGrouping,
    e.country,
    SUM(CAST(e.totalTransactionRevenue AS UNSIGNED)) AS total_revenue
FROM ecommerce_sessions e
INNER JOIN Country_Revenue cr
    ON e.country = cr.country
WHERE e.totalTransactionRevenue IS NOT NULL
GROUP BY e.channelGrouping, e.country
ORDER BY total_revenue DESC;
'''

In [12]:
# Execute the query and load the result into a Pandas DataFrame
channel_analysis = pd.read_sql(query, conn)

# Show the result
print(channel_analysis)

   channelGrouping        country  total_revenue
0         Referral  United States   5.596654e+10
1   Organic Search  United States   2.744382e+10
2           Direct  United States   1.387820e+10
3   Organic Search      Venezuela   9.952160e+09
4          Display  United States   8.497860e+09
5   Organic Search         Canada   4.719760e+09
6      Paid Search  United States   4.190910e+09
7         Referral         Canada   3.204460e+09
8         Referral         Taiwan   7.971000e+08
9           Social  United States   4.646600e+08
10          Direct         Canada   2.183400e+08
11  Organic Search        Curaçao   2.083300e+08
12          Direct      Venezuela   9.250000e+07


In [13]:
# Query for Test Case 2: User Behavior Analysis
query = '''
WITH User_Metrics AS (
    SELECT 
        fullVisitorId,
        AVG(CAST(timeOnSite AS UNSIGNED)) AS avg_timeOnSite,
        AVG(CAST(pageviews AS UNSIGNED)) AS avg_pageviews,
        AVG(CAST(sessionQualityDim AS UNSIGNED)) AS avg_sessionQualityDim
    FROM ecommerce_sessions
    WHERE timeOnSite IS NOT NULL AND pageviews IS NOT NULL
    GROUP BY fullVisitorId
),
Average_Metrics AS (
    SELECT 
        AVG(avg_timeOnSite) AS overall_avg_timeOnSite,
        AVG(avg_pageviews) AS overall_avg_pageviews
    FROM User_Metrics
)

SELECT 
    u.fullVisitorId,
    u.avg_timeOnSite,
    u.avg_pageviews,
    u.avg_sessionQualityDim
FROM User_Metrics u, Average_Metrics a
WHERE u.avg_timeOnSite > a.overall_avg_timeOnSite
AND u.avg_pageviews < a.overall_avg_pageviews;
'''

In [14]:
# Execute the query and load the result into a Pandas DataFrame
product_performance = pd.read_sql(query, conn)

# Show the result
print(product_performance)

           fullVisitorId  avg_timeOnSite  avg_pageviews  avg_sessionQualityDim
0    7313828956068851679           927.0           11.0                   63.0
1    6036794406403793540          1616.0           13.0                   38.0
2     997271511212133268           743.0           19.0                    NaN
3    6238727055806281579           647.0            5.0                    NaN
4    7732587129685157308           701.0           10.0                    NaN
..                   ...             ...            ...                    ...
592   127472362838253777           940.0           15.0                    NaN
593  9979229817860302312          1661.0           13.0                    NaN
594  1419607020881916790          1804.0            7.0                    NaN
595  4811553119148587033          1175.0           16.0                    NaN
596  7929960947613776663           765.0           11.0                    NaN

[597 rows x 4 columns]


In [15]:
# Query for Test Case 3: Product Performance
from sqlalchemy import text

query = text('''
WITH Product_Sales AS (
    SELECT 
        v2ProductName,
        SUM(CAST(productRevenue AS UNSIGNED)) AS total_revenue,
        SUM(CAST(productQuantity AS UNSIGNED)) AS total_quantity_sold,
        SUM(CAST(productRefundAmount AS UNSIGNED)) AS total_refund
    FROM ecommerce_sessions
    WHERE productRevenue IS NOT NULL
    GROUP BY v2ProductName
)

SELECT 
    v2ProductName,
    total_revenue,
    total_quantity_sold,
    total_refund,
    (total_revenue - total_refund) AS net_revenue,
    CASE 
        WHEN total_refund > (total_revenue * 0.1) THEN 'Refund > 10%'
        ELSE 'Refund <= 10%'
    END AS refund_flag
FROM Product_Sales
ORDER BY net_revenue DESC;
''')

In [16]:
# Execute the query and load the result into a Pandas DataFrame
product_performance = pd.read_sql(query, conn)

# Show the result
print(product_performance)

                         v2ProductName  total_revenue  total_quantity_sold  \
0             Suitcase Organizer Cubes            0.0                  NaN   
1   26 oz Double Wall Insulated Bottle            0.0                  NaN   
2             Metal Texture Roller Pen            0.0                  NaN   
3                             Maze Pen            0.0                  NaN   
4             Gunmetal Roller Ball Pen            0.0                  NaN   
..                                 ...            ...                  ...   
79               Reusable Shopping Bag            0.0                 22.0   
80                     Chevron Shopper            0.0                  4.0   
81      Google Zipper-front Sports Bag            0.0                  1.0   
82                  Waterpoof Gear Bag            0.0                  1.0   
83                           Sport Bag            0.0                 44.0   

   total_refund net_revenue    refund_flag  
0          None   

In [17]:
conn.close()